<a href="https://colab.research.google.com/github/nickwan/nwds-stream-notebooks/blob/main/2025_savant_stance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# font

In [ ]:
def increase_font():
  from IPython.display import Javascript
  display(Javascript('''
  for (rule of document.styleSheets[0].cssRules){
    if (rule.selectorText=='body') {
      rule.style.fontSize = '24px'
      break
    }
  }
  '''))
get_ipython().events.register('pre_run_cell', increase_font)

# new savant data?

In [ ]:
!pip install pybaseball
!pip install catboost
!pip install -U pandas

<IPython.core.display.Javascript object>

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pybaseball as bb
import catboost as cb
from sklearn.metrics import mean_squared_error, log_loss
from sklearn.model_selection import KFold
project_dir = 'drive/My Drive/2025-stance'

<IPython.core.display.Javascript object>

In [ ]:
df = bb.statcast(start_dt='2024-04-01',end_dt='2024-07-01')
df.shape

<IPython.core.display.Javascript object>

This is a large query, it may take a moment to complete


/usr/local/lib/python3.11/dist-packages/pybaseball/statcast.py:50: UserWarning: 
That's a nice request you got there. It'd be a shame if something were to happen to it.
We strongly recommend that you enable caching before running this. It's as simple as `pybaseball.cache.enable()`.
Since the Statcast requests can take a *really* long time to run, if something were to happen, like: a disconnect;
gremlins; computer repair by associates of Rudy Giuliani; electromagnetic interference from metal trash cans; etc.;
you could lose a lot of progress. Enabling caching will allow you to immediately recover all the successful
subqueries if that happens.
  warnings.warn(_OVERSIZE_WARNING)
  0%|          | 0/92 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/pybaseball/datahelpers/postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_cop

(351579, 113)

In [ ]:
df.columns.values

<IPython.core.display.Javascript object>

array(['pitch_type', 'game_date', 'release_speed', 'release_pos_x',
       'release_pos_z', 'player_name', 'batter', 'pitcher', 'events',
       'description', 'spin_dir', 'spin_rate_deprecated',
       'break_angle_deprecated', 'break_length_deprecated', 'zone', 'des',
       'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type',
       'hit_location', 'bb_type', 'balls', 'strikes', 'game_year',
       'pfx_x', 'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b',
       'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y',
       'tfs_deprecated', 'tfs_zulu_deprecated', 'umpire', 'sv_id', 'vx0',
       'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot',
       'hit_distance_sc', 'launch_speed', 'launch_angle',
       'effective_speed', 'release_spin_rate', 'release_extension',
       'game_pk', 'fielder_2', 'fielder_3', 'fielder_4', 'fielder_5',
       'fielder_6', 'fielder_7', 'fielder_8', 'fielder_9',
       'release_pos_y', 'estimated_ba_using_speedangle',
  

In [ ]:
df.head()

<IPython.core.display.Javascript object>

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,n_thruorder_pitcher,n_priorpa_thisgame_player_at_bat,pitcher_days_since_prev_game,batter_days_since_prev_game,pitcher_days_until_next_game,batter_days_until_next_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle
664,SL,2024-07-01,83.0,-2.68,5.71,"Payamps, Joel",595909,606303,single,hit_into_play,...,1,4,2,1,3,2,3.31,-0.7,0.7,28.8
697,FF,2024-07-01,94.4,-2.44,5.52,"Payamps, Joel",669911,606303,field_out,hit_into_play,...,1,4,2,1,3,1,1.61,0.57,-0.57,28.4
713,FF,2024-07-01,94.8,-2.51,5.55,"Payamps, Joel",669911,606303,NaN,foul,...,1,4,2,1,3,1,1.39,0.55,-0.55,28.5
745,SL,2024-07-01,81.9,-2.73,5.76,"Payamps, Joel",669911,606303,NaN,swinging_strike,...,1,4,2,1,3,1,3.62,-0.56,0.56,28.0
780,SL,2024-07-01,81.9,-2.72,5.68,"Payamps, Joel",669911,606303,NaN,ball,...,1,4,2,1,3,1,3.52,-0.34,0.34,32.6
